In [ ]:
#pip uninstall numpy
#pip install numpy

In [1]:
# Imports

# Import Classification Parameters
from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report
# Import warnings filter
import warnings
# Data Reader
import pandas as pd
# Train and Test split
from sklearn.model_selection import train_test_split
#Models
from sklearn.linear_model import LogisticRegression
# Plot
import matplotlib.pyplot as plt
# Import of Numpy
import numpy

%matplotlib inline
%pip install pomegranate

import matplotlib.pyplot as plt
import seaborn; seaborn.set_style('whitegrid')

from pomegranate import *

numpy.random.seed(0)
numpy.set_printoptions(suppress=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.3 MB 8.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pomegranate: filename=pomegranate-0.14.8-cp37-cp37m-linux_x86_64.whl size=15068165 sha256=0a70b97d7d8e08df724a5fd2f1a9bd4943de59b96e1940ed29a7972011fc7d6f
  Stored in directory: /root/.cache/pip/wheels/24/68/69/0eaab474ef1d65abedcd47de8a38ab21d221d329954d7edd24
Successfully built pomegranate


In [2]:
# ignore all future warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)
warnings.filterwarnings('ignore',category=Warning)

In [3]:
# Import the data set
df = pd.read_csv('dataset.csv')
X = df.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]].values # Aqui temos as features
y = df.iloc[:, -1].values # Aqui temos a condição


# Nova secção

In [4]:
# hidden_layer (condition) |  v1,v2,v3
#            1             |    000
#            0             |    121
#           ...            |    ...

hidden_layer = df.iloc[:, -1].values # Aqui temos as hidden layers
print("Shape da Hidden Layer: ", hidden_layer.shape)

obs_imp = df.iloc[:, [1,11,12]].values # Aqui temos as observation layers com maior importancia
print("Shape da Observation Layer: ", obs_imp.shape)

Shape da Hidden Layer:  (296,)
Shape da Observation Layer:  (296, 3)


In [5]:
#########################################   Logistic Regression  #############################################################

# Data Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Model and Fitting
classifier = LogisticRegression(C=0.1,penalty='l2',solver='newton-cg')# best parameters from LogReg_best_parameters.py
classifier.fit(X_train, y_train)

importance = classifier.coef_[0]
# summarize feature importance
for i,obs_imp in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,obs_imp))
    
# As features mais importantes estão com TAB:

# Feature: 0, Score: 0.00499 -> Que é: age[numbers]
    # Feature: 1, Score: 0.49400 -> Que é: sex[0, 1] -> Categorização: 0 = female; 1 = male
# Feature: 2, Score: 0.28289 -> Que é: cp[1, 2, 3, 4]
# Feature: 3, Score: 0.01929 -> Que é: trestbps[numbers]
# Feature: 4, Score: 0.00386 -> Que é: chol[numbers]
# Feature: 5, Score: -0.09167 -> Que é: fbs[0, 1]
# Feature: 6, Score: 0.19371 -> Que é: restecg[0, 1, 2]
# Feature: 7, Score: -0.02304 -> Que é: thalach[numbers]
# Feature: 8, Score: 0.26940 -> Que é: exang[0, 1]
# Feature: 9, Score: 0.34504 -> Que é: oldpeak[numbers]
# Feature: 10, Score: 0.31220 -> Que é: slope[1, 2, 3]
    # Feature: 11, Score: 0.68902 -> Que é: ca [0,1,2,3] -> 
    # Feature: 12, Score: 0.61421 -> Que é: thal [0,1,2] -> Categorização: 0 = normal; 1 = fixed defect; 2 = reversable defect

Feature: 0, Score: 0.00499
Feature: 1, Score: 0.49400
Feature: 2, Score: 0.28289
Feature: 3, Score: 0.01929
Feature: 4, Score: 0.00386
Feature: 5, Score: -0.09167
Feature: 6, Score: 0.19371
Feature: 7, Score: -0.02304
Feature: 8, Score: 0.26940
Feature: 9, Score: 0.34504
Feature: 10, Score: 0.31220
Feature: 11, Score: 0.68902
Feature: 12, Score: 0.61421


In [6]:
# Probabilidades Hidden Layers

# São as variáveis que vão contar quantas vezes aparecem na coluna das hidden layers:
count01 = 0; # Aparece 01 seguidos
count10 = 0; # Aparece 10 seguidos
count00 = 0; # Aparece 00 seguidos
count11 = 0; # Aparece 10 seguidos

# São as variáveis que vão contar quantas vezes aparecem na coluna das hidden layers:
count1 = 0; # Aparece 1's seguidos
count0 = 0; # Aparece 0's seguidos

# Ciclo for que vai percorrer os índices da coluna das hidden layers.
for i in range(0,295):
    if(hidden_layer[i]==0 and hidden_layer[i+1]==1): # Aparece 01 seguidos
        count01 += 1;
    elif(hidden_layer[i]==1 and hidden_layer[i+1]==0): # Aparece 10 seguidos
        count10 += 1;
    elif(hidden_layer[i]==0 and hidden_layer[i+1]==0): # Aparece 00 seguidos
        count00 += 1;
    elif(hidden_layer[i]==1 and hidden_layer[i+1]==1): # Aparece 10 seguidos
        count11 += 1;
        
# Ciclo for que vai percorrer os índices da coluna das hidden layers.
for j in range(0,296):
    if(hidden_layer[j]==0): # Aparece 0
        count0 += 1;
    elif(hidden_layer[j]==1): # Aparece 1
        count1 += 1;

n_elem_comp_hiddenLayer = count00+count10+count01+count11

print("---''---")

print("Número de vezes que aparece 0: ",count0)
probabilidade_0 = count0 / (n_elem_comp_hiddenLayer + 1)
print("A sua probabilidade: %.2f" % probabilidade_0)
print("Número de vezes que aparece 1: ",count1)
probabilidade_1 = count1 / (n_elem_comp_hiddenLayer + 1)
print("A sua probabilidade: %.2f" % probabilidade_1)

print("---''---")
print("Número de vezes que aparece a sequencia 00: ",count00)
print("Número de vezes que aparece a sequencia 01: ",count01)
print("Número de vezes que aparece a sequencia 10: ",count10)
print("Número de vezes que aparece a sequencia 11: ",count11)
print("Número de elementos de comparação para hiddenLayer: ",n_elem_comp_hiddenLayer)

print("---''---")
n_total = 295
probabilidade_00 = count00 / (count00 + count01)
probabilidade_01 = count01 / (count00 + count01)
probabilidade_10 = count10 / (count11 + count10)
probabilidade_11 = count11 / (count11 + count10)

print("Probabilidade de 00: %.2f" % probabilidade_00)
print("Probabilidade de 01: %.2f" % probabilidade_01)
print("Probabilidade de 10: %.2f" % probabilidade_10)
print("Probabilidade de 11: %.2f" % probabilidade_11)

---''---
Número de vezes que aparece 0:  159
A sua probabilidade: 0.54
Número de vezes que aparece 1:  137
A sua probabilidade: 0.46
---''---
Número de vezes que aparece a sequencia 00:  107
Número de vezes que aparece a sequencia 01:  52
Número de vezes que aparece a sequencia 10:  51
Número de vezes que aparece a sequencia 11:  85
Número de elementos de comparação para hiddenLayer:  295
---''---
Probabilidade de 00: 0.67
Probabilidade de 01: 0.33
Probabilidade de 10: 0.38
Probabilidade de 11: 0.62


In [7]:
dkl1 = {}
dlk1 = {}

letter = '@'
df1 = pd.read_csv('label1_trans.csv',sep=',')[['sex','ca','thal']]
print(df1)

df1['key'] = ""
df1['letter'] = ""

for idx,row in df1.iterrows():
    key = str(row['sex']) + ":" + str(row['ca']) + ":" + str(row['thal'])
    
    if(key not in dkl1.keys()):
        letter = chr(ord(letter)+1)
        dkl1[key] = letter
        L = letter
    else:
        L = dkl1[key]
        
    dlk1[L] = key    
    df1['key'] = key
    df1['letter'] = L

print(dkl1)
print(dlk1)

     sex  ca  thal
0      1   1     0
1      1   1     0
2      1   2     0
3      1   0     2
4      1   0     0
..   ...  ..   ...
133    1   0     2
134    1   0     2
135    1   0     2
136    1   0     2
137    1   0     2

[138 rows x 3 columns]
{'1:1:0': 'A', '1:2:0': 'B', '1:0:2': 'C', '1:0:0': 'D', '1:3:1': 'E', '1:1:2': 'F', '0:2:0': 'G', '0:1:0': 'H', '1:3:2': 'I', '0:1:2': 'J', '1:1:1': 'K', '1:2:2': 'L', '1:3:0': 'M', '0:2:2': 'N', '0:3:2': 'O', '1:2:1': 'P', '0:0:0': 'Q', '0:3:0': 'R', '0:0:2': 'S', '0:2:1': 'T', '1:0:1': 'U'}
{'A': '1:1:0', 'B': '1:2:0', 'C': '1:0:2', 'D': '1:0:0', 'E': '1:3:1', 'F': '1:1:2', 'G': '0:2:0', 'H': '0:1:0', 'I': '1:3:2', 'J': '0:1:2', 'K': '1:1:1', 'L': '1:2:2', 'M': '1:3:0', 'N': '0:2:2', 'O': '0:3:2', 'P': '1:2:1', 'Q': '0:0:0', 'R': '0:3:0', 'S': '0:0:2', 'T': '0:2:1', 'U': '1:0:1'}


In [8]:
# Vamos buscar os observáveis que aparecem quando a hidden layer é = 1
dTrans1 = {}
hidden_layer_is_1 = pd.read_csv('label1.csv')
hidden_layer1 = hidden_layer_is_1.iloc[:, [0,1, 2]].values

print("Observáveis quando a Hidden Layer é igual a 1.\n")

(unique1, counts1) = numpy.unique(hidden_layer1, return_counts=True, axis=0)

probabilidade1 = 0
probabilidade_total1 = 0
probabilidades_de_1 = []

for i in range(len(unique1)):
    key = ':'.join([str(x) for x in unique1[i]])
    letter = dkl1[key]
    
    print("Observável:", unique1[i],)
    print("Aparições:", counts1[i])
    probabilidade1 = counts1[i] / count1
    probabilidades_de_1.append(probabilidade1)
    print("Probabilidade: (%d/%d) = %.5f\n" % (counts1[i], count1, probabilidade1))
    probabilidade_total1 = probabilidade_total1 + probabilidade1
    
    dTrans1[letter] = probabilidade1
    
print("\nA probabildiade total: %.1f" % probabilidade_total1)
print("Existem ", len(unique1), "obseváveis no total.")

print(dTrans1)

Observáveis quando a Hidden Layer é igual a 1.

Observável: [0 0 0]
Aparições: 6
Probabilidade: (6/137) = 0.04380

Observável: [0 0 2]
Aparições: 4
Probabilidade: (4/137) = 0.02920

Observável: [0 1 0]
Aparições: 1
Probabilidade: (1/137) = 0.00730

Observável: [0 1 2]
Aparições: 2
Probabilidade: (2/137) = 0.01460

Observável: [0 2 0]
Aparições: 3
Probabilidade: (3/137) = 0.02190

Observável: [0 2 1]
Aparições: 1
Probabilidade: (1/137) = 0.00730

Observável: [0 2 2]
Aparições: 4
Probabilidade: (4/137) = 0.02920

Observável: [0 3 0]
Aparições: 1
Probabilidade: (1/137) = 0.00730

Observável: [0 3 2]
Aparições: 3
Probabilidade: (3/137) = 0.02190

Observável: [1 0 0]
Aparições: 7
Probabilidade: (7/137) = 0.05109

Observável: [1 0 1]
Aparições: 2
Probabilidade: (2/137) = 0.01460

Observável: [1 0 2]
Aparições: 26
Probabilidade: (26/137) = 0.18978

Observável: [1 1 0]
Aparições: 11
Probabilidade: (11/137) = 0.08029

Observável: [1 1 1]
Aparições: 4
Probabilidade: (4/137) = 0.02920

Observável

In [9]:

dkl = {}
dlk = {}

letter = '@'
df = pd.read_csv('label0_trans.csv',sep=',')[['sex','ca','thal']]
print(df)

df['key'] = ""
df['letter'] = ""

for idx,row in df.iterrows():
    key = str(row['sex']) + ":" + str(row['ca']) + ":" + str(row['thal'])
    
    if(key not in dkl.keys()):
        letter = chr(ord(letter)+1)
        dkl[key] = letter
        L = letter
    else:
        L = dkl[key]
        
    dlk[L] = key    
    df['key'] = key
    df['letter'] = L

print(dkl)
print(dlk)

     sex  ca  thal
0      1   1     0
1      0   2     0
2      0   0     0
3      1   0     0
4      1   0     2
..   ...  ..   ...
155    1   0     0
156    1   0     2
157    0   0     0
158    1   0     0
159    0   0     0

[160 rows x 3 columns]
{'1:1:0': 'A', '0:2:0': 'B', '0:0:0': 'C', '1:0:0': 'D', '1:0:2': 'E', '1:0:1': 'F', '1:2:0': 'G', '0:1:0': 'H', '1:1:2': 'I', '0:0:2': 'J', '1:3:2': 'K', '1:3:0': 'L'}
{'A': '1:1:0', 'B': '0:2:0', 'C': '0:0:0', 'D': '1:0:0', 'E': '1:0:2', 'F': '1:0:1', 'G': '1:2:0', 'H': '0:1:0', 'I': '1:1:2', 'J': '0:0:2', 'K': '1:3:2', 'L': '1:3:0'}


In [10]:
# Vamos buscar os observáveis que aparecem quando a hidden layer é = 0
dTrans0 = {}
hidden_layer_is_0 = pd.read_csv('label0.csv')
hidden_layer0 = hidden_layer_is_0.iloc[:, [0,1, 2]].values

print("Observáveis quando a Hidden Layer é igual a 0.\n")

(unique0, counts0) = numpy.unique(hidden_layer0, return_counts=True, axis=0)

probabilidade0 = 0
probabilidade_total0 = 0
probabilidades_de_0 = []

for i in range(len(unique0)):
    key = ':'.join([str(x) for x in unique0[i]])
    letter = dkl[key]
    
    print("Observável:", unique0[i],)
    print("Aparições:", counts0[i])
    probabilidade0 = counts0[i] / count0
    probabilidades_de_0.append(probabilidade0)
    print("Probabilidade: (%d/%d) = %.5f\n" % (counts0[i], count0, probabilidade0))
    probabilidade_total0 = probabilidade_total0 + probabilidade0
    
    dTrans0[letter] = probabilidade0
    
print("\nA probabildiade total: %.1f" % probabilidade_total0)
print("Existem ", len(unique0), "obseváveis no total.")

print(dTrans0)




Observáveis quando a Hidden Layer é igual a 0.

Observável: [0 0 0]
Aparições: 52
Probabilidade: (52/159) = 0.32704

Observável: [0 0 2]
Aparições: 2
Probabilidade: (2/159) = 0.01258

Observável: [0 1 0]
Aparições: 12
Probabilidade: (12/159) = 0.07547

Observável: [0 2 0]
Aparições: 5
Probabilidade: (5/159) = 0.03145

Observável: [1 0 0]
Aparições: 50
Probabilidade: (50/159) = 0.31447

Observável: [1 0 1]
Aparições: 6
Probabilidade: (6/159) = 0.03774

Observável: [1 0 2]
Aparições: 19
Probabilidade: (19/159) = 0.11950

Observável: [1 1 0]
Aparições: 4
Probabilidade: (4/159) = 0.02516

Observável: [1 1 2]
Aparições: 4
Probabilidade: (4/159) = 0.02516

Observável: [1 2 0]
Aparições: 2
Probabilidade: (2/159) = 0.01258

Observável: [1 3 0]
Aparições: 1
Probabilidade: (1/159) = 0.00629

Observável: [1 3 2]
Aparições: 2
Probabilidade: (2/159) = 0.01258


A probabildiade total: 1.0
Existem  12 obseváveis no total.
{'C': 0.3270440251572327, 'J': 0.012578616352201259, 'H': 0.07547169811320754, 

In [11]:
d0 = DiscreteDistribution({'A': 0.025157232704402517, 'B': 0.031446540880503145, 'C': 0.3270440251572327, 'D' : 0.31446540880503143,
                           'E':0.11949685534591195, 'F' : 0.03773584905660377, 'G' : 0.012578616352201259,'H': 0.07547169811320754,
                           'J': 0.025157232704402517,'K': 0.012578616352201259, 'L': 0.012578616352201259, 'P': 0.006289308176100629})

d1 = DiscreteDistribution({'Q': 0.043795620437956206, 'S': 0.029197080291970802, 'H': 0.0072992700729927005, 'J': 0.014598540145985401, 
                            'G': 0.021897810218978103, 'T': 0.0072992700729927005, 'N': 0.029197080291970802, 'R': 0.0072992700729927005, 
                            'O': 0.021897810218978103, 'D': 0.051094890510948905, 'U': 0.014598540145985401, 'C': 0.1897810218978102, 
                            'A': 0.08029197080291971, 'K': 0.029197080291970802, 'F': 0.1897810218978102, 'B': 0.029197080291970802, 
                            'P': 0.021897810218978103, 'L': 0.11678832116788321, 'M': 0.029197080291970802, 'E': 0.014598540145985401, 
                            'I': 0.051094890510948905})


s0 = State(d0, name='non-disease')
s1 = State(d1, name='disease')

In [14]:
model = HiddenMarkovModel()
model.add_states( [s0, s1] )
model.add_transition( model.start, s0, 0.46)
model.add_transition( model.start, s1, 0.54)
model.add_transition( s0, s1,probabilidade_10)
model.add_transition( s1,s0, probabilidade_01)
model.add_transition( s1,s1, probabilidade_00)
model.add_transition( s0,s0, probabilidade_11)
model.bake()

In [32]:
seq0 = 'QSJNONJH'

seq = numpy.array(list(seq0))

hmm_predictions = model.predict(seq)

print("sequence: {}".format(''.join(seq)))
print("hmm pred: {}".format(''.join(map( str, hmm_predictions))))

print(model.predict_proba(seq))

sequence: QSJNONJH
hmm pred: 00000011
[[1.         0.        ]
 [1.         0.        ]
 [0.68181277 0.31818723]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [0.41454518 0.58545482]
 [0.10091589 0.89908411]]
